In [ ]:
import sqlite3
import requests
from datetime import datetime, timedelta, date,

In [15]:
datetime(2025,7,15)

datetime.datetime(2025, 7, 15, 0, 0)

In [23]:
"""Fetch prayer times for a date range from the new API and store each day in DB."""
url = f"https://api.aladhan.com/v1/calendarByCity/from/{datetime(2025,7,15).strftime('%d-%m-%Y')}/to/{datetime(2025,7,30).strftime('%d-%m-%Y')}"

params = {
    "city": 'Chicago',
    "country": 'United States',
    "method": 2,    # Same method as single-day
    "school": 1,
    "calendarMethod": "HJCoSA",
    "iso8601": "false",
    # add more params if needed
}

response = requests.get(url, params=params)
response.raise_for_status()
data = response.json()

In [28]:
data['data'][0]['date']

{'readable': '14 Jul 2025',
 'timestamp': '1752550501',
 'gregorian': {'date': '14-07-2025',
  'format': 'DD-MM-YYYY',
  'day': '14',
  'weekday': {'en': 'Monday'},
  'month': {'number': 7, 'en': 'July'},
  'year': '2025',
  'designation': {'abbreviated': 'AD', 'expanded': 'Anno Domini'},
  'lunarSighting': False},
 'hijri': {'date': '19-01-1447',
  'format': 'DD-MM-YYYY',
  'day': '19',
  'weekday': {'en': 'Al Athnayn', 'ar': 'الاثنين'},
  'month': {'number': 1, 'en': 'Muḥarram', 'ar': 'مُحَرَّم', 'days': 30},
  'year': '1447',
  'designation': {'abbreviated': 'AH', 'expanded': 'Anno Hegirae'},
  'holidays': ['Urs of Shaykh Darwish Muhammad ق'],
  'adjustedHolidays': [],
  'method': 'HJCoSA'}}

In [ ]:
hijri_date = data['data'][0]['date']['hijri']['date']
print(hijri_date, hijri_month_name, hijri_month_number)

19-01-1447 Muḥarram 1


In [24]:
for data_obj in data['data']:
    date_str = data_obj['date']['gregorian']['date']
    hijri_date = data_obj['date']['hijri']['date']
    timings = data_obj['timings']
    print(f"Date: {date_str}, Hijri: {hijri_date}, Timings: {timings}")


Date: 14-07-2025, Hijri: 19-01-1447, Timings: {'Fajr': '03:49 (CDT)', 'Sunrise': '05:28 (CDT)', 'Dhuhr': '12:57 (CDT)', 'Asr': '18:09 (CDT)', 'Sunset': '20:25 (CDT)', 'Maghrib': '20:25 (CDT)', 'Isha': '22:03 (CDT)', 'Imsak': '03:39 (CDT)', 'Midnight': '00:56 (CDT)', 'Firstthird': '23:26 (CDT)', 'Lastthird': '02:27 (CDT)'}
Date: 15-07-2025, Hijri: 20-01-1447, Timings: {'Fajr': '03:51 (CDT)', 'Sunrise': '05:29 (CDT)', 'Dhuhr': '12:57 (CDT)', 'Asr': '18:09 (CDT)', 'Sunset': '20:24 (CDT)', 'Maghrib': '20:24 (CDT)', 'Isha': '22:02 (CDT)', 'Imsak': '03:41 (CDT)', 'Midnight': '00:57 (CDT)', 'Firstthird': '23:26 (CDT)', 'Lastthird': '02:27 (CDT)'}
Date: 16-07-2025, Hijri: 21-01-1447, Timings: {'Fajr': '03:52 (CDT)', 'Sunrise': '05:30 (CDT)', 'Dhuhr': '12:57 (CDT)', 'Asr': '18:08 (CDT)', 'Sunset': '20:23 (CDT)', 'Maghrib': '20:23 (CDT)', 'Isha': '22:01 (CDT)', 'Imsak': '03:42 (CDT)', 'Midnight': '00:57 (CDT)', 'Firstthird': '23:26 (CDT)', 'Lastthird': '02:28 (CDT)'}
Date: 17-07-2025, Hijri: 22-

In [2]:
   
def fetch_prayer_times_range(start_date, end_date, city, country=""):
    """Fetch prayer times for a date range from the new API and store each day in DB."""
    url = f"https://api.aladhan.com/v1/calendarByCity/from/{start_date.strftime('%d-%m-%Y')}/to/{end_date.strftime('%d-%m-%Y')}"
    
    params = {
        "city": city,
        "country": country,
        "method": 2,    # Same method as single-day
        "school": 1,
        "calendarMethod": "HJCoSA",
        "iso8601": "false",
        # add more params if needed
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # The API returns data in data.data array for each day
        for day_data in data.get("data", []):
            greg_date_str = day_data["date"]["gregorian"]["date"]  # e.g. "01-02-2025"
            date = datetime.strptime(greg_date_str, "%d-%m-%Y").date()
            timings = day_data["timings"]
            
            times = {
                "Fajr": convert_to_24hr(clean_timezone_suffix(timings["Fajr"])),
                "Dhuhr": convert_to_24hr(clean_timezone_suffix(timings["Dhuhr"])),
                "Asr": convert_to_24hr(clean_timezone_suffix(timings["Asr"])),
                "Maghrib": convert_to_24hr(clean_timezone_suffix(timings["Maghrib"])),
                "Isha": convert_to_24hr(clean_timezone_suffix(timings["Isha"]))
            }

            store_prayer_times(date, city, times)
        print(f"Stored prayer times for {city} on between {start_date} and {end_date} successfully.")

        return True
    except Exception as e:
        print(f"Error fetching date range prayer times: {e}")
        return False


In [ ]:


DB_PATH = "prayer_times.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prayer_times (
            date TEXT,
            city TEXT,
            fajr TEXT,
            dhuhr TEXT,
            asr TEXT,
            maghrib TEXT,
            isha TEXT,
            PRIMARY KEY (date, city)
        )
    """)
    conn.commit()
    conn.close()

def store_prayer_times(date, city, times):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR REPLACE INTO prayer_times (date, city, fajr, dhuhr, asr, maghrib, isha)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        date.strftime("%Y-%m-%d"),
        city,
        times["Fajr"],
        times["Dhuhr"],
        times["Asr"],
        times["Maghrib"],
        times["Isha"]
    ))
    conn.commit()
    conn.close()
def get_prayer_times_from_db(date, city):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        SELECT fajr, dhuhr, asr, maghrib, isha FROM prayer_times
        WHERE date=? AND city=?
    """, (date.strftime("%Y-%m-%d"), city))
    row = cursor.fetchone()
    conn.close()
    if row:
        return {
            "Fajr": row[0],
            "Dhuhr": row[1],
            "Asr": row[2],
            "Maghrib": row[3],
            "Isha": row[4]
        }
    return None
def get_prayer_times_range_from_db(start_date, end_date, city):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT date, fajr, dhuhr, asr, maghrib, isha
        FROM prayer_times
        WHERE city = ? AND date BETWEEN ? AND ?
    ''', (city, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")))

    rows = cursor.fetchall()
    conn.close()

    return {row[0]: {
        "Fajr": row[1], "Dhuhr": row[2], "Asr": row[3],
        "Maghrib": row[4], "Isha": row[5]
    } for row in rows}
